In [22]:
import requests
import pandas as pd
import itertools
import time
import numpy as np

import os
from dotenv import load_dotenv

In [23]:
load_dotenv()
BASE_URL = "https://api3.natst.at"
API_KEY = os.getenv("NAT_API_KEY")

In [17]:
def ncaab_list_all_dates_for_season(league, year):
    url = f"{BASE_URL}/{API_KEY}/games/{league}/{year}"
    print(url)

    # initialize set
    dates = set()
    res = requests.get(url).json()
    for key, value in res['games'].items():
        dates.add(value['gameday'])
    next_page_url = res['meta'].get('page-next', False)
    
    # loop through pages
    while(next_page_url is not False):
        print(next_page_url)
        res = requests.get(next_page_url).json()
        for key, value in res['games'].items():
            dates.add(value['gameday'])
        next_page_url = res['meta'].get('page-next', False)
    
    return dates

def ncaab_get_pbp_by_date(league, date):
    url = f"{BASE_URL}/{API_KEY}/playbyplay/{league}/{date}"
    print(url)

    # initialize df
    df = pd.DataFrame()
    res = requests.get(url).json()
    for key, value in res['playbyplay'].items():
        df_temp = pd.json_normalize(value)
        df = pd.concat([df, df_temp])
    next_page_url = res['meta'].get('page-next', False)

    # loop through pages
    while(next_page_url is not False):
        print(next_page_url)
        res = requests.get(next_page_url).json()
        for key, value in res['playbyplay'].items():
            df_temp = pd.json_normalize(value)
            df = pd.concat([df, df_temp])
        next_page_url = res['meta'].get('page-next', False)

    return df

def ncaab_get_list_of_game_codes_from_single_day_pbp(df):
    return df['game.code'].unique().tolist()

def ncaab_get_final_score(df, game_code):
    last_row = df[df['game.code'] == game_code].tail(1)
    home_team = last_row.iloc[0]['game.home']
    away_team = last_row.iloc[0]['game.visitor']
    winner = home_team if int(last_row.iloc[0]['game.score-home']) > int(last_row.iloc[0]['game.score-vis']) else away_team
    return {
        'gameTitle': last_row.iloc[0]['game.description'],
        'date': last_row.iloc[0]['game.gameday'],
        'home': home_team,
        'away': away_team,
        'winner': winner,
        'winnerScore': max(last_row.iloc[0]['game.score-home'], last_row.iloc[0]['game.score-vis']),
        'loseScore': min(last_row.iloc[0]['game.score-home'], last_row.iloc[0]['game.score-vis'])
    }

def ncaab_clean_pbp_df(df: pd.DataFrame):
    mapping = {
        'Atlanta': 'Atlanta Hawks',
        'Boston': 'Boston Celtics',
        'Brooklyn': 'Brooklyn Nets',
        'Charlotte': 'Charlotte Hornets',
        'Chicago': 'Chicago Bulls',
        'Cleveland': 'Cleveland Cavaliers',
        'Dallas': 'Dallas Mavericks',
        'Denver': 'Denver Nuggets',
        'Detroit': 'Detroit Pistons',
        'Golden State': 'Golden State Warriors',
        'Houston': 'Houston Rockets',
        'Indiana': 'Indiana Pacers',
        'L.A. Clippers': 'Los Angeles Clippers',
        'L.A. Lakers': 'Los Angeles Lakers',
        'Memphis': 'Memphis Grizzlies',
        'Miami': 'Miami Heat',
        'Milwaukee': 'Milwaukee Bucks',
        'Minnesota': 'Minnesota Timberwolves',
        'New Orleans': 'New Orleans Pelicans',
        'New York': 'New York Knicks',
        'Oklahoma City': 'Oklahoma City Thunder',
        'Orlando': 'Orlando Magic',
        'Philadelphia': 'Philadelphia 76ers',
        'Phoenix': 'Phoenix Suns',
        'Portland': 'Portland Trail Blazers',
        'Sacramento': 'Sacramento Kings',
        'San Antonio': 'San Antonio Spurs',
        'Toronto': 'Toronto Raptors',
        'Utah': 'Utah Jazz',
        'Washington': 'Washington Wizards'
    }
    df['game.visitor'] = df['game.visitor'].map(mapping).fillna(df['game.visitor'])
    df['game.home'] = df['game.home'].map(mapping).fillna(df['game.home'])

def ncaab_get_deficit_time(df, game_code):
    box_score = ncaab_get_final_score(df, game_code)
    # print(box_score)
    winning_team = box_score['winner']

    df_game = df[df['game.code'] == game_code]
    df_winner = df_game[df_game['team.team'] == winning_team]
    # print(df_winner)
    
    deficit = min(pd.to_numeric(df_winner['thediff'], errors='coerce', downcast='integer'))
    df_deficit = df_winner[df_winner['thediff'] == deficit].tail(1)
    if len(df_deficit)<1:
         df_deficit = df_winner[df_winner['thediff'] == "+"+str(deficit)].tail(1)
    if len(df_deficit)<1:
        df_deficit = df_winner[df_winner['thediff'] == str(deficit)].tail(1)
    return {
        'game title': box_score['gameTitle'],
        'date': box_score['date'],
        'winning_team': winning_team,
        'deficit': deficit,
        'period': df_deficit.iloc[0]['game.period'],
        'clock': df_deficit.iloc[0]['game.time'],
        'seconds_remaining': ncaab_get_seconds_remaining(df_deficit.iloc[0]['game.period'], df_deficit.iloc[0]['game.time'])
    }

def ncaab_get_seconds_remaining(period, clock: str):
    try: 
        minutes, seconds, ms = clock.split(':')
    except Exception:
        minutes, seconds = clock.split(":")
        ms = 0
    if int(period) < 2:
        return (2-int(period))*20*60 + int(minutes)*60 + float(seconds) + float(ms)/100
    else:
        return int(minutes)*60 + float(seconds) + float(ms)/100

In [4]:
active_game_dates = ncaab_list_all_dates_for_season("MBB", 2024)
active_game_dates

https://api3.natst.at/9384-3abba6/games/MBB/2024
https://api3.natst.at/9384-3abba6/games/mbb/2024/100
https://api3.natst.at/9384-3abba6/games/mbb/2024/200
https://api3.natst.at/9384-3abba6/games/mbb/2024/300
https://api3.natst.at/9384-3abba6/games/mbb/2024/400
https://api3.natst.at/9384-3abba6/games/mbb/2024/500
https://api3.natst.at/9384-3abba6/games/mbb/2024/600
https://api3.natst.at/9384-3abba6/games/mbb/2024/700
https://api3.natst.at/9384-3abba6/games/mbb/2024/800
https://api3.natst.at/9384-3abba6/games/mbb/2024/900
https://api3.natst.at/9384-3abba6/games/mbb/2024/1000
https://api3.natst.at/9384-3abba6/games/mbb/2024/1100
https://api3.natst.at/9384-3abba6/games/mbb/2024/1200
https://api3.natst.at/9384-3abba6/games/mbb/2024/1300
https://api3.natst.at/9384-3abba6/games/mbb/2024/1400
https://api3.natst.at/9384-3abba6/games/mbb/2024/1500
https://api3.natst.at/9384-3abba6/games/mbb/2024/1600
https://api3.natst.at/9384-3abba6/games/mbb/2024/1700
https://api3.natst.at/9384-3abba6/games/mb

{'2023-11-06',
 '2023-11-07',
 '2023-11-08',
 '2023-11-09',
 '2023-11-10',
 '2023-11-11',
 '2023-11-12',
 '2023-11-13',
 '2023-11-14',
 '2023-11-15',
 '2023-11-16',
 '2023-11-17',
 '2023-11-18',
 '2023-11-19',
 '2023-11-20',
 '2023-11-21',
 '2023-11-22',
 '2023-11-23',
 '2023-11-24',
 '2023-11-25',
 '2023-11-26',
 '2023-11-27',
 '2023-11-28',
 '2023-11-29',
 '2023-11-30',
 '2023-12-01',
 '2023-12-02',
 '2023-12-03',
 '2023-12-04',
 '2023-12-05',
 '2023-12-06',
 '2023-12-07',
 '2023-12-08',
 '2023-12-09',
 '2023-12-10',
 '2023-12-11',
 '2023-12-12',
 '2023-12-13',
 '2023-12-14',
 '2023-12-15',
 '2023-12-16',
 '2023-12-17',
 '2023-12-18',
 '2023-12-19',
 '2023-12-20',
 '2023-12-21',
 '2023-12-22',
 '2023-12-23',
 '2023-12-24',
 '2023-12-27',
 '2023-12-28',
 '2023-12-29',
 '2023-12-30',
 '2023-12-31',
 '2024-01-01',
 '2024-01-02',
 '2024-01-03',
 '2024-01-04',
 '2024-01-05',
 '2024-01-06',
 '2024-01-07',
 '2024-01-08',
 '2024-01-09',
 '2024-01-10',
 '2024-01-11',
 '2024-01-12',
 '2024-01-

In [5]:
len(active_game_dates)

148

In [70]:
# continue querying
already_queried_dates = set(df['game.gameday'])
remaining = active_game_dates - already_queried_dates
len(remaining)

80

In [7]:
df = pd.DataFrame()
df = pd.read_csv('outputs/full_filtered_ncaab_output.csv')

# for date in remaining:
#     print(f"Querying for day: {date}...")
#     single_day_pbp_df = ncaab_get_pbp_by_date("MBB", date)
#     df = pd.concat([df, single_day_pbp_df])
#     print("Successfully added day. Sleeping for 0s...")
# df.to_csv('outputs/full_ncaab_output.csv')

C:\Users\cwpac\AppData\Local\Temp\ipykernel_11492\2264859695.py:2: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('outputs/full_filtered_ncaab_output.csv')


In [63]:
df = df[~df['tags'].str.contains('MISC', na=False)]
df.to_csv('outputs/full_filtered_ncaab_output.csv')

In [8]:
sorted(df['game.home'].unique().tolist())

['Abilene Christian Wildcats',
 'Air Force Falcons',
 'Akron Zips',
 'Alabama A&M Bulldogs',
 'Alabama Crimson Tide',
 'Alabama State Hornets',
 'Albany Great Danes',
 'Alcorn State Braves',
 'American Eagles',
 'Appalachian State Mountaineers',
 'Arizona State Sun Devils',
 'Arizona Wildcats',
 'Arkansas Razorbacks',
 'Arkansas State Red Wolves',
 'Arkansas-Little Rock Trojans',
 'Arkansas-Pine Bluff Golden Lions',
 'Army Black Knights',
 'Auburn Tigers',
 'Austin Peay Governors',
 'BYU Cougars',
 'Ball State Cardinals',
 'Baylor Bears',
 'Bellarmine Knights',
 'Belmont Bruins',
 'Bethesda (CA)',
 'Bethune-Cookman Wildcats',
 'Binghamton Bearcats',
 'Boise State Broncos',
 'Boston College Eagles',
 'Boston University Terriers',
 'Bowling Green Falcons',
 'Bradley Braves',
 'Brown Bears',
 'Bryant Bulldogs',
 'Bucknell Bison',
 'Buffalo Bulls',
 'Butler Bulldogs',
 'Cal Poly Mustangs',
 'Cal State Bakersfield Roadrunners',
 'Cal State Fullerton Titans',
 'Cal State Northridge Matadors'

In [12]:
print(len(ncaab_get_list_of_game_codes_from_single_day_pbp(df)))
df.columns

2700


Index(['Unnamed: 0', 'id', 'explanation', 'scoringplay', 'tags', 'thediff',
       'players.primary.code', 'players.primary.name', 'team.code',
       'team.team', 'opponent.opponent_id', 'opponent.opponent', 'game.code',
       'game.gameday', 'game.description', 'game.visitor', 'game.visitor-code',
       'game.score-vis', 'game.home', 'game.home-code', 'game.score-home',
       'game.period', 'game.time', 'game.sequence', 'game.onfloorvis',
       'game.onfloorhome', 'venue.code', 'venue.name', 'venue.location',
       'venue.longitude', 'venue.latitude', 'game.playoffs'],
      dtype='object')

In [18]:
comebacks = []
for game in ncaab_get_list_of_game_codes_from_single_day_pbp(df):
    if game == 1258358:
        continue
    print(game)
    output = ncaab_get_deficit_time(df, game)
    comebacks.append(output)

1245791
1245877
1246094
1246441
1247309
1247492
1247612
1247920
1248479
1248503
1248933
1249305
1249351
1249514
1249647
1250774
1258635
1245780
1246721
1247456
1247867
1248111
1248270
1248362
1248516
1248732
1249068
1249715
1249760
1249970
1250265
1250794
1245403
1245713
1245741
1245770
1245861
1246149
1246233
1246366
1246392
1246473
1246598
1246922
1247056
1247107
1247428
1247450
1247644
1248014
1248188
1248210
1248264
1248803
1248990
1249156
1249216
1249436
1249718
1249841
1250071
1250481
1250585
1258652
1258745
1258746
1258753
1258755
1258761
1258768
1258771
1258780
1258781
1258794
1258807
1259183
1259196
1259217
1259219
1259221
1259223
1259229
1259230
1259233
1259235
1259236
1259242
1245400
1245424
1245680
1245710
1245767
1245798
1245826
1245858
1245884
1245947
1245970
1245997
1246022
1246049
1246078
1246097
1246146
1246177
1246204
1246230
1246255
1246285
1246313
1246390
1246419
1246499
1246525
1246550
1246595
1246623
1246654
1246684
1246707
1246758
1246785
1246874
1246898
1246920


In [19]:
comebacks_df = pd.DataFrame(comebacks)
exclude = [
    "Canisius Golden Griffins vs Wofford Terriers", 
    "Colorado Buffaloes vs Miami (Fla.) Hurricanes"
]
comebacks_df = comebacks_df[~comebacks_df['game title'].isin(exclude)]
comebacks_df['analytical_lead_minus_five_abs'] = comebacks_df['deficit'].apply(lambda x: min(x+5, 0))
comebacks_df['analytical_time_required'] = (comebacks_df['analytical_lead_minus_five_abs'])**2
comebacks_df['analytical_diff'] = comebacks_df['analytical_time_required'] - comebacks_df['seconds_remaining']
comebacks_df

,game title,date,winning_team,deficit,period,clock,seconds_remaining,analytical_lead_minus_five_abs,analytical_time_required,analytical_diff
0,Alabama A&M Bulldogs at Prairie View Panthers,2024-01-29,Prairie View Panthers,-5,1,12:19:00,1939.0,0,0,-1939.0
1,Alcorn State Braves at Bethune-Cookman Wildcats,2024-01-29,Alcorn State Braves,-4,2,03:14:00,194.0,0,0,-194.0
2,Arkansas-Pine Bluff Golden Lions at Grambling ...,2024-01-29,Arkansas-Pine Bluff Golden Lions,-2,1,18:26:00,2306.0,0,0,-2306.0
3,Boston University Terriers at Holy Cross Crusa...,2024-01-29,Holy Cross Crusaders,-12,1,06:48:00,1608.0,-7,49,-1559.0
4,Howard Bison at Coppin State Eagles,2024-01-29,Howard Bison,-5,1,17:02:00,2222.0,0,0,-2222.0
...,...,...,...,...,...,...,...,...,...,...
2694,Middle Tennessee Blue Raiders vs Louisiana Tec...,2024-03-13,Middle Tennessee Blue Raiders,-8,1,04:13:00,1453.0,-3,9,-1444.0
2695,Cincinnati Bearcats vs Kansas Jayhawks,2024-03-13,Cincinnati Bearcats,0,2,03:54:00,234.0,0,0,-234.0
2696,Missouri Tigers vs Georgia Bulldogs,2024-03-13,Georgia Bulldogs,-8,2,12:37:00,757.0,-3,9,-748.0
2697,Arizona State Sun Devils vs Utah Utes,2024-03-13,Utah Utes,0,1,18:22:00,2302.0,0,0,-2302.0


In [21]:
comebacks_df.to_csv('outputs/ncaab_comebacks_v1.csv')

## Investigation

In [84]:
df_error = df[df['game.code'] == "1258358"]
df_error['game.description']

0    Colorado Buffaloes vs Miami (Fla.) Hurricanes
0    Colorado Buffaloes vs Miami (Fla.) Hurricanes
0    Colorado Buffaloes vs Miami (Fla.) Hurricanes
0    Colorado Buffaloes vs Miami (Fla.) Hurricanes
0    Colorado Buffaloes vs Miami (Fla.) Hurricanes
                         ...                      
0    Colorado Buffaloes vs Miami (Fla.) Hurricanes
0    Colorado Buffaloes vs Miami (Fla.) Hurricanes
0    Colorado Buffaloes vs Miami (Fla.) Hurricanes
0    Colorado Buffaloes vs Miami (Fla.) Hurricanes
0    Colorado Buffaloes vs Miami (Fla.) Hurricanes
Name: game.description, Length: 470, dtype: object